# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 20
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr20.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr20.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 187 187

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000101255,ENSG00000125826,ENSG00000101266,ENSG00000101280,ENSG00000125818,ENSG00000101298,ENSG00000088832,ENSG00000101307,ENSG00000089012,ENSG00000198053
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000101255,1.00000000,0.07116780,-0.01661844,0.12590226,0.06163206,0.04033341,0.0656184145,-0.08399902,0.03417133,0.09272230
ENSG00000125826,0.07116780,1.00000000,-0.04990565,0.08615582,-0.02007391,-0.07842001,0.0007446513,0.45129085,-0.07229110,-0.09455175
ENSG00000101266,-0.01661844,-0.04990565,1.00000000,-0.15247166,-0.10278400,-0.09379928,-0.1203321645,-0.05763548,-0.07945995,-0.05885954
ENSG00000101280,0.12590226,0.08615582,-0.15247166,1.00000000,-0.03173951,0.07143994,-0.0462339328,0.04168030,-0.04259196,0.06605722
ENSG00000125818,0.06163206,-0.02007391,-0.10278400,-0.03173951,1.00000000,0.04809711,-0.2797302935,0.01121849,-0.04626598,-0.03855960
ENSG00000101298,0.04033341,-0.07842001,-0.09379928,0.07143994,0.04809711,1.00000000,-0.0539865198,0.14891914,-0.01919014,-0.13382007


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 14

[1] -0.03507483 -0.12330615 -0.13757894 -0.18160785 -0.19528941 -0.22878618
 [7] -0.30163887 -0.41968304 -0.55389187 -0.65147067 -0.67843320 -1.00869220
[13] -1.07086471 -2.73749059

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 187 187

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000101255,ENSG00000125826,ENSG00000101266,ENSG00000101280,ENSG00000125818,ENSG00000101298,ENSG00000088832,ENSG00000101307,ENSG00000089012,ENSG00000198053
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000101255,1.00000000,0.07116780,-0.01661844,0.12590226,0.06163206,0.04033341,0.0656184145,-0.08399902,0.03417133,0.09272230
ENSG00000125826,0.07116780,1.00000000,-0.04990565,0.08615582,-0.02007391,-0.07842001,0.0007446513,0.45129085,-0.07229110,-0.09455175
ENSG00000101266,-0.01661844,-0.04990565,1.00000000,-0.15247166,-0.10278400,-0.09379928,-0.1203321645,-0.05763548,-0.07945995,-0.05885954
ENSG00000101280,0.12590226,0.08615582,-0.15247166,1.00000000,-0.03173951,0.07143994,-0.0462339328,0.04168030,-0.04259196,0.06605722
ENSG00000125818,0.06163206,-0.02007391,-0.10278400,-0.03173951,1.00000000,0.04809711,-0.2797302935,0.01121849,-0.04626598,-0.03855960
ENSG00000101298,0.04033341,-0.07842001,-0.09379928,0.07143994,0.04809711,1.00000000,-0.0539865198,0.14891914,-0.01919014,-0.13382007


,ENSG00000101255,ENSG00000125826,ENSG00000101266,ENSG00000101280,ENSG00000125818,ENSG00000101298,ENSG00000088832,ENSG00000101307,ENSG00000089012,ENSG00000198053
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000101255,1.000000000,0.043315543,-0.008469768,0.06083937,0.038120981,0.009123620,0.02305991,-0.048594232,-0.0258213228,0.052929224
ENSG00000125826,0.043315543,1.000000000,-0.021832068,0.05290155,-0.002175285,0.004453637,0.01862289,0.140991757,0.0038127294,-0.064178282
ENSG00000101266,-0.008469768,-0.021832068,1.000000000,-0.04084000,-0.024473111,-0.032017804,-0.06786354,-0.049237382,-0.0241530739,-0.001132488
ENSG00000101280,0.060839366,0.052901553,-0.040840002,1.00000000,-0.018761835,0.071250866,-0.01801663,0.003632707,0.0097449884,0.039432014
ENSG00000125818,0.038120981,-0.002175285,-0.024473111,-0.01876183,1.000000000,0.043822260,-0.09110818,0.010378850,-0.0005461786,-0.018106840
ENSG00000101298,0.009123620,0.004453637,-0.032017804,0.07125087,0.043822260,1.000000000,-0.02949655,0.049233109,-0.0118557875,-0.039780304


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)